<a href="https://colab.research.google.com/github/Murilo1514/Diabets_prediction_pipeline/blob/main_merge/SVM_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install optuna
!pip install ucimlrepo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo
from functools import partial
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import *
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline





# Importando a Base de dados e separando 5%



In [44]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

data = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

data["Diabetes_binary"] = aux["Diabetes_binary"]

#print(cdc_diabetes_health_indicators.metadata)

#print(cdc_diabetes_health_indicators.variables)

data.columns

<ipython-input-44-0eb90a80a6c7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

In [45]:
data['Age']=data['Age'].astype('int64')
data['Education']=data['Education'].astype('int64')
data['Income']=data['Income'].astype('int64')
data['BMI']=data['BMI'].astype('int64')
data['CholCheck']=data['CholCheck'].astype('int64')
data['Smoker']=data['Smoker'].astype('int64')
data['Stroke']=data['Stroke'].astype('int64')
data['HeartDiseaseorAttack']=data['HeartDiseaseorAttack'].astype('int64')
data['PhysActivity']=data['PhysActivity'].astype('int64')
data['Fruits']=data['Fruits'].astype('int64')
data['Veggies']=data['Veggies'].astype('int64')
data['HvyAlcoholConsump']=data['HvyAlcoholConsump'].astype('int64')
data['AnyHealthcare']=data['AnyHealthcare'].astype('int64')
data['NoDocbcCost']=data['NoDocbcCost'].astype('int64')
data['GenHlth']=data['GenHlth'].astype('int64')
data['MentHlth']=data['MentHlth'].astype('int64')
data['PhysHlth']=data['PhysHlth'].astype('int64')
data['DiffWalk']=data['DiffWalk'].astype('int64')
data['Sex']=data['Sex'].astype('int64')
data['Diabetes_binary']=data['Diabetes_binary'].astype('int64')
data['HighBP']=data['HighBP'].astype('int64')
data['HighChol']=data['HighChol'].astype('int64')

<ipython-input-45-bc92d329addf>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-bc92d329addf>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-bc92d329addf>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-bc92d32

In [46]:
data.duplicated().sum()

data.drop_duplicates(inplace=True)

<ipython-input-46-1ecf93f2f3f5>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']


In [48]:
data1 = data.sample(frac=0.05, random_state=42)

x = data1.drop('Diabetes_binary', axis = 1)
y = data1['Diabetes_binary']


In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15, random_state=42, stratify=y_train)


In [50]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [61]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        #('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('bin', 'passthrough', binary_columns)
    ])


pipeline = Pipeline(steps=[
      #('outlier_remover', outlier_remover),
      ('preprocessor_svm', preprocessor_svm),
      #('svm', SVC()),
      ('svm' , SVC(kernel='sigmoid', gamma = 0.01, C=733)),
      ('pca',pca),
      ('model', model)
  ])


###                                Oversampling

In [52]:
over_methods = [
    ADASYN(random_state=7),
    BorderlineSMOTE(random_state=7),
    RandomOverSampler(random_state=7),
    SMOTE(random_state=7),
    SVMSMOTE(random_state=7)
]


In [53]:
classifiers =[
    SVC()
]



Metricas para comparar os métodos

In [54]:
metrics = {
    'roc_auc': 'roc_auc',
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

In [32]:
steps = [('model', SVC(kernel='sigmoid', C=733, gamma= 0.01  ))]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

for metric_name, metric in metrics.items():
    scores = cross_val_score(
        pipeline, x_train, y_train, scoring=metric, cv=cv, n_jobs=-1
    )
    print(f"{metric_name.upper()} for SVM is {round(np.mean(scores), 5)}")



ROC_AUC for SVM is 0.5806
ACCURACY for SVM is 0.78292
PRECISION for SVM is 0.30066
RECALL for SVM is 0.2897
F1 for SVM is 0.2948


In [164]:
for classifier in classifiers:
    for method in over_methods:
        X_resampled, y_resampled = method.fit_resample(X, y)

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', classifier)
        ])

        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

        for metric_name, metric in metrics.items():
            scores = cross_val_score(
                pipeline, X_resampled, y_resampled, scoring=metric, cv=cv, n_jobs=-1
            )
            print(f"{metric_name.upper()} for {classifier.__class__.__name__} and {method.__class__.__name__} is {round(np.mean(scores), 5)}")



ROC_AUC for SVC and ADASYN is 0.86367
ACCURACY for SVC and ADASYN is 0.78555
PRECISION for SVC and ADASYN is 0.76707
RECALL for SVC and ADASYN is 0.81827
F1 for SVC and ADASYN is 0.79182
ROC_AUC for SVC and BorderlineSMOTE is 0.89333
ACCURACY for SVC and BorderlineSMOTE is 0.81564
PRECISION for SVC and BorderlineSMOTE is 0.78962
RECALL for SVC and BorderlineSMOTE is 0.86064
F1 for SVC and BorderlineSMOTE is 0.82357
ROC_AUC for SVC and RandomOverSampler is 0.8769
ACCURACY for SVC and RandomOverSampler is 0.8007
PRECISION for SVC and RandomOverSampler is 0.77309
RECALL for SVC and RandomOverSampler is 0.85131
F1 for SVC and RandomOverSampler is 0.81029
ROC_AUC for SVC and SMOTE is 0.87505
ACCURACY for SVC and SMOTE is 0.79776
PRECISION for SVC and SMOTE is 0.78069
RECALL for SVC and SMOTE is 0.82831
F1 for SVC and SMOTE is 0.80376
ROC_AUC for SVC and SVMSMOTE is 0.90517
ACCURACY for SVC and SVMSMOTE is 0.8292
PRECISION for SVC and SVMSMOTE is 0.81418
RECALL for SVC and SVMSMOTE is 0.8531

In [55]:
method = BorderlineSMOTE(random_state=7)
x_train_resampled, y_train_resampled = method.fit_resample(x_train, y_train)

x_train, y_train = x_train_resampled, y_train_resampled

In [56]:
x_train.shape

(13160, 21)

## Testando acurácia antes de aplicar o PCA

In [57]:
column_names = numerical_columns + categorical_columns + binary_columns

x_train = pd.DataFrame(x_train, columns=column_names)
x_test = pd.DataFrame(x_test, columns=column_names)
x_validation = pd.DataFrame(x_validation, columns=column_names)

In [67]:
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia com PCA ({21} componentes): {accuracy}')

Acurácia com PCA (21 componentes): 0.693681917211329


In [66]:
for i in range(1, 22):
    print("==========================================================")

    # Criando o PCA com o número de componentes 'i'
    pca = PCA(n_components=i)

    # Criando o pipeline com o PCA e o modelo SVC
    pipeline = make_pipeline(pca, model)

    # Treinando o pipeline
    pipeline.fit(x_train, y_train)

    # Fazendo previsões
    y_pred = pipeline.predict(x_test)

    # Calculando a acurácia
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Acurácia com PCA ({i} componentes): {accuracy}')

Acurácia com PCA (1 componentes): 0.5947712418300654
Acurácia com PCA (2 componentes): 0.6762527233115468
Acurácia com PCA (3 componentes): 0.6549019607843137
Acurácia com PCA (4 componentes): 0.665359477124183
Acurácia com PCA (5 componentes): 0.6618736383442266
Acurácia com PCA (6 componentes): 0.677124183006536
Acurácia com PCA (7 componentes): 0.6723311546840959
Acurácia com PCA (8 componentes): 0.6854030501089324
Acurácia com PCA (9 componentes): 0.6671023965141613
Acurácia com PCA (10 componentes): 0.6915032679738562
Acurácia com PCA (11 componentes): 0.6901960784313725
Acurácia com PCA (12 componentes): 0.6901960784313725
Acurácia com PCA (13 componentes): 0.6941176470588235
Acurácia com PCA (14 componentes): 0.6880174291938997
Acurácia com PCA (15 componentes): 0.6867102396514161
Acurácia com PCA (16 componentes): 0.6849673202614379
Acurácia com PCA (17 componentes): 0.690631808278867
Acurácia com PCA (18 componentes): 0.6932461873638345
Acurácia com PCA (19 componentes): 0.694

In [ ]:
def objective(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])

    C = trial.suggest_categorical('C', [c for c in range(1, 1000, 1)])
    gamma = trial.suggest_float('gamma', 0.01, 2, step=0.005)
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])

    pipeline.set_params(svm__C=C, svm__gamma=gamma, svm__kernel=kernel)

    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_validation)
    score = f1_score(y_validation, y_pred)

    return score

# Criar e otimizar o estudo
study = optuna.create_study(direction='maximize')
study.optimize(objective)

# Exibir os melhores hiperparâmetros
print(study.best_params)

In [67]:
pipeline = Pipeline([
      #('outlier_remover', outlier_remover),
      ('preprocessor_svm', preprocessor_svm),
      ('svm', SVC())
  ])

C = 900
gamma = 0.11215038461268562
kernel=  'sigmoid'

pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_temp)
score = f1_score(y_temp, y_pred)
print(score)
print(classification_report(y_temp, y_pred))


SyntaxError: incomplete input (<ipython-input-67-81cfc10a30d3>, line 1)

In [119]:
print(score)

0.1935483870967742


In [14]:
print(confusion_matrix(y_temp, y_pred))



[[124   8]
 [ 17   3]]
